# Mark 1 Transformer
This is to test building and training a transformer from scratch. I have been using the Towards Data Science article: [Build Your Own Transformer](https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb) as a starting point.

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
from datasets import Dataset
from tqdm import tqdm

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output
    

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
    

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [3]:
class GPTLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(GPTLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [6]:
class GPT(nn.Module):
    def __init__(self, num_layers, num_heads, d_model, d_ff, dropout):
        super(GPT, self).__init__()
        self.layers = nn.ModuleList([GPTLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        
    def forward(self, x, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, src_mask, tgt_mask)
        return x
    

Now we need to read in the test data to get training started.

In [11]:
# read in the test data
test_wiki_embeddings_back = Dataset.load_from_disk('test_wiki2_embeddings')

# define the model
model = GPT(num_layers=2, num_heads=8, d_model=1024, d_ff=2048, dropout=0.1)

# use the data to train the model
model.train()

# define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

# train the model
for epoch in range(10):
    for batch in tqdm(test_wiki_embeddings_back):
        # get the batch
        batch = batch['embeddings']
        # split the batch into input and target
        input_batch = batch[:-1]
        target_batch = batch[-1]
        # zero the gradients
        optimizer.zero_grad()
        # forward pass
        outputs = model(input_batch, None, target_batch)
        # compute the loss
        loss = loss_fn(outputs.view(-1, outputs.size(-1)), target_batch.view(-1))
        # backward pass
        loss.backward()
        # update the parameters
        optimizer.step()   

# now save the model
torch.save(model.state_dict(), 'test_gpt_model.pt')





  0%|          | 0/4 [00:00<?, ?it/s]


TypeError: linear(): argument 'input' (position 1) must be Tensor, not list